In [40]:
import os

import shutil

import numpy as np

from PIL import Image

from tensorflow.keras.preprocessing import image

from tensorflow.keras.models import load_model

from tensorflow.keras.applications import ResNet50


In [39]:
from PIL import UnidentifiedImageError
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
import zipfile
import shutil
# Load the pretrained model
model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add your own classification head on top of the ResNet50 base
x = GlobalAveragePooling2D()(model.output)
x = Dense(1024, activation='relu')(x)
num_classes=3
output = Dense(num_classes, activation='softmax')(x)

# Path to your unlabeled dataset ZIP file
zip_file_path = "/content/shoe_boot_sandal.zip"

# Create a temporary directory to extract the contents of the ZIP file
extracted_dir = "/content/temp_extracted_images"
os.makedirs(extracted_dir, exist_ok=True)

# Extract the contents of the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir)

# Create folders for each predicted class
class_names = ['boot', 'sandal', 'shoe']

for class_name in class_names:
    output_dir = "/content/classified_images"
    class_dir = os.path.join(output_dir, class_name)
    os.makedirs(class_dir, exist_ok=True)

# Iterate through the images in the dataset
for root, _, files in os.walk(extracted_dir):
    for file in files:
        image_path = os.path.join(root, file)

        try:
            # Load and preprocess the image
            img = image.load_img(image_path, target_size=(224, 224))
            x = image.img_to_array(img)
            x = np.expand_dims(x, axis=0)
            x = preprocess_input(x)

            # Make predictions using the pretrained model
            predictions = model.predict(x)
            predicted_class_index = np.argmax(predictions)
            predicted_class_name = class_names[predicted_class_index]


            # Move the image to the corresponding class folder
            destination_dir = os.path.join(output_dir, predicted_class_name)
            shutil.move(image_path, destination_dir)

        except (UnidentifiedImageError, OSError):
            print(f"Skipped file: {image_path} - Not a valid image file")

# Clean up the temporary extracted directory
shutil.rmtree(extracted_dir)


Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 194ms/step


In [41]:
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_28 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_28[0][0]']               
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           